# Putting it all together

So far we have done the following on the prior Notebooks:

- **Notebook 01**: We loaded the Azure Search Engine with enriched PDFs in index: "cogsrch-index-files"
- **Notebook 02**: We loaded more information to the Search Engine this time using a CSV file with 90k rows/articles in index: "cogsrch-index-csv"
- **Notebook 03**: We added AzureOpenAI GPT models to enhance the the production of the answer by using Utility Chains of LLMs
- **Notebook 04**: We manually loaded an index with large/complex PDFs information , "cogsrch-index-books-vector"
- **Notebook 05**: We added memory to our system in order to power a conversational Chat Bot
- **Notebook 06**: We introduced Agents and Tools and built the first Skill/Agent, that can do RAG over a search engine
- **Notebook 07**: We build a second Agent (Pandas) in order to be able to solve a more complex task: ask questions to Tabular datasets
- **Notebook 08**: We used a SQL Agent in order to talk to a SQL Database directly
- **Notebook 09**: We used another  Agent in order to talk to the Bing Search API and create a Bing Chat Clone and implemented callbacks for real-time streaming and tool information
- **Notebook 10**: We built an API Agent that can translate a question into the right API calls, giving us the capability to talk to any datasource that provides a RESTFul API.


We are missing one more thing: **How do we glue all these features together into a very smart GPT Smart Search Engine Chat Bot?**

We want a virtual assistant for our company that can get the question, think what tool to use, then get the answer. The goal is that, regardless of the source of the information (Search Engine, Bing Search, SQL Database, CSV File, JSON File, APIs, etc), the Assistant can answer the question correctly using the right tool.

In this Notebook we are going to create that "brain" Agent (also called Master Agent), that:

1) understands the question, interacts with the user 
2) talks to other specialized Agents that are connected to diferent sources
3) once it get's the answer it delivers it to the user or let the specialized Agent to deliver it directly

This is the same concept of [AutoGen](https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/): Agents talking to each other.

![image](https://www.microsoft.com/en-us/research/uploads/prod/2023/09/AutoGen_Fig1.png)

In [2]:
import os
import random
import json
import requests
from operator import itemgetter
from typing import Union, List
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec, ConfigurableField
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

#custom libraries that we will use later in the app
from common.utils import (
    DocSearchAgent, 
    CSVTabularAgent, 
    SQLSearchAgent, 
    ChatGPTTool, 
    BingSearchAgent, 
    APISearchAgent, 
    reduce_openapi_spec
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT 

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))


In [6]:
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Get the Tools - DocSearch Agent, CSV Agent, SQL Agent, Web Search Agent, ChatGPT, API Agent

**Consider the following concept:** Agents, which are essentially software entities designed to perform specific tasks, can be equipped with tools. These tools themselves can be other agents, each possessing their own set of tools. This creates a layered structure where tools can range from code sequences to human actions, forming interconnected chains. Ultimately, you're constructing a network of agents and their respective tools, all collaboratively working towards solving a specific task (This is what ChatGPT is). This network operates by leveraging the unique capabilities of each agent and tool, creating a dynamic and efficient system for task resolution.

In the file `common/utils.py` we created Agent Tools Classes for each of the Functionalities that we developed in prior Notebooks. 

In [7]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

# We can run the everything with GPT3.5, but try also GPT4 and see the difference in the quality of responses
# You will notice that GPT3.5 is not as reliable when using multiple sources.

llm = AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], 
                      temperature=0, max_tokens=COMPLETION_TOKENS)

# Uncomment below if you want to see the answers streaming
# llm = AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager)


In [11]:
doc_indexes = ["cogsrch-index-files", "cogsrch-index-csv"]
doc_search = DocSearchAgent(llm=llm, indexes=doc_indexes,
                           k=6, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="docsearch",
                           description="useful when the questions includes the term: docsearch",
                           callback_manager=cb_manager, verbose=False)

In [10]:
book_indexes = ["cogsrch-index-books"]
book_search = DocSearchAgent(llm=llm, indexes=book_indexes,
                           k=10, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="booksearch",
                           description="useful when the questions includes the term: booksearch",
                           callback_manager=cb_manager, verbose=False)

In [9]:
# BingSearchAgent is a langchain Tool class to use the Bing Search API (https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)
www_search = BingSearchAgent(llm=llm, k=5, callback_manager=cb_manager, 
                             name="bing",
                             description="useful when the questions includes the term: bing",
                             verbose=False)

In [8]:
## CSVTabularAgent is a custom Tool class crated to Q&A over CSV files
file_url = "./data/all_states/all-states-history.csv"
csv_search = CSVTabularAgent(path=file_url, llm=llm, callback_manager=cb_manager,
                             name="csvfile",
                             description="useful when the questions includes the term: csvfile",
                             verbose=False)

In [12]:
## SQLDbAgent is a custom Tool class created to Q&A over a MS SQL Database
sql_search = SQLSearchAgent(llm=llm, k=30, callback_manager=cb_manager,
                            name="sqlsearch",
                            description="useful when the questions includes the term: sqlsearch",
                            verbose=False)

In [13]:
## ChatGPTTool is a custom Tool class created to talk to ChatGPT knowledge
chatgpt_search = ChatGPTTool(llm=llm, callback_manager=cb_manager,
                             name="chatgpt",
                            description="useful when the questions includes the term: chatgpt",
                            verbose=False)

In [14]:
## APISearchAgent is a custom Tool class created to talk to any API 

spec = json.load(open("./data/api_specs/kraken.json", encoding='utf-8'))

api_search = APISearchAgent(llm=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=1000),
                            llm_search=AzureChatOpenAI(deployment_name=os.environ["GPT4_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=1000),
                            api_spec=str(reduce_openapi_spec(spec)),
                            callback_manager=cb_manager,
                            name="apisearch",
                            description="useful when the questions includes the term: apisearch",
                            verbose=False)

### Variables/knobs to use for customization

As you have seen so far, there are many knobs that you can dial up or down in order to change the behavior of your GPT Smart Search engine application, these are the variables you can tune:

- <u>llm</u>:
  - **deployment_name**: this is the deployment name of your Azure OpenAI model. This of course dictates the level of reasoning and the amount of tokens available for the conversation. 
  - **temperature**: How creative you want your responses to be
  - **max_tokens**: How long you want your responses to be. It is recommended a minimum of 500
- <u>Tools</u>: To each tool you can add the following parameters to modify the defaults (set in utils.py), these are very important since they are part of the system prompt and determines what tool to use and when.
  - **name**: the name of the tool
  - **description**: when the brain agent should use this tool
- <u>DocSearchAgent</u>: 
  - **k**: The top k results per index from the text search action
  - **similarity_k**: top k results combined from the vector search action
  - **reranker_th**: threshold of the semantic search reranker. Picks results that are above the threshold. Max possible score=4
- <u>BingSearchAgent</u>:
  - **k**: The top k results from the bing search action
- <u>SQLSearchAgent</u>:
  - **k**: The top k results from the SQL search action. Adds TOP clause to the query
  
in `utils.py` you can also tune:
- <u>model_tokens_limit</u>: In this function you can edit what is the maximum allows of tokens reserve for the content. Remember that the remaining will be for the system prompt plus the answer

### Test the Tools

In [16]:
# Test the Document Search Tool with a question that we know it has the answer for
printmd(await doc_search.arun("How Covid affects obese people? and elderly?"))

Tool: docsearch
Agent Action: 
Invoking: `docsearch` with `{'query': 'Covid effects on obese people'}`



Agent Action: 
Invoking: `docsearch` with `{'query': 'Covid effects on elderly people'}`





### Impact of COVID-19 on Obese People

Obesity has been identified as a significant risk factor for severe outcomes in COVID-19 patients. Obese individuals are more likely to experience severe symptoms, require hospitalization, and have a higher risk of mortality. The reasons for this increased risk include:

1. **Impaired Immune Response**: Obesity can impair the immune system, making it less effective at fighting off infections, including COVID-19.
2. **Chronic Inflammation**: Obesity is associated with chronic low-grade inflammation, which can exacerbate the inflammatory response to COVID-19, leading to more severe symptoms.
3. **Respiratory Issues**: Excess weight can restrict lung function and make it more difficult for obese individuals to breathe, which is particularly problematic with a respiratory illness like COVID-19.
4. **Comorbidities**: Obese individuals often have other health conditions such as diabetes, hypertension, and cardiovascular diseases, which can complicate COVID-19 treatment and recovery [[1]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1143779/?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-01-30T14:55:42Z&st=2024-07-10T06:55:42Z&spr=https&sig=NQgAnpUqrSUPKdOZwtvdSQP2pjwoeUK0xlNtd%2F554t8%3D).

### Impact of COVID-19 on Elderly People

Elderly individuals are particularly vulnerable to COVID-19 due to several factors:

1. **Weakened Immune System**: Aging is associated with a decline in immune function, making it harder for elderly individuals to fight off infections.
2. **Higher Prevalence of Comorbidities**: Older adults are more likely to have chronic health conditions such as heart disease, diabetes, and respiratory diseases, which can worsen the prognosis of COVID-19.
3. **Increased Risk of Severe Outcomes**: Studies have shown that elderly patients are more likely to experience severe symptoms, require intensive care, and have higher mortality rates compared to younger populations.
4. **Impact of Social Isolation**: Measures to reduce contact, such as social distancing and lockdowns, can have significant mental health impacts on the elderly, potentially exacerbating feelings of loneliness and depression [[2]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2876165/?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-01-30T14:55:42Z&st=2024-07-10T06:55:42Z&spr=https&sig=NQgAnpUqrSUPKdOZwtvdSQP2pjwoeUK0xlNtd%2F554t8%3D).

In summary, both obese and elderly individuals face heightened risks from COVID-19 due to a combination of physiological vulnerabilities and the presence of comorbid conditions. It is crucial for these populations to take extra precautions and for healthcare systems to prioritize their protection and treatment.

In [17]:
# Test the Bing Search Agent
printmd(await www_search.arun("Who are the family member names of the current president of India?"))

Tool: bing
Agent Action: 
Invoking: `Searcher` with `{'query': 'current president of India family members names'}`



Agent Action: 
Invoking: `Searcher` with `{'query': 'Droupadi Murmu family members names'}`



Agent Action: 
Invoking: `WebFetcher` with `{'url': 'https://starsunfolded.com/droupadi-murmu/'}`



Agent Action: 
Invoking: `Searcher` with `{'query': 'Droupadi Murmu family members names site:starsunfolded.com'}`





The current President of India, Droupadi Murmu, has the following family members:

1. **Husband**: Shyam Charan Murmu (deceased) [[1]](https://starsunfolded.com/droupadi-murmu/).
2. **Daughter**: Itishree Murmu [[2]](https://starsunfolded.com/itishree-murmu/).

Droupadi Murmu has faced significant personal losses, including the deaths of her husband and two sons within a span of six years [[1]](https://starsunfolded.com/droupadi-murmu/).

For more detailed information, you can visit the [Stars Unfolded page on Droupadi Murmu](https://starsunfolded.com/droupadi-murmu/).

In [18]:
# Test the ChatGPTWrapper Search Tool
printmd(await chatgpt_search.arun("what is the function in python that allows me to get a random number?"))

Tool: chatgpt


En Python, puedes obtener un número aleatorio utilizando la biblioteca `random`. Esta biblioteca proporciona varias funciones para generar números aleatorios. La función más comúnmente utilizada para obtener un número aleatorio dentro de un rango específico es `random.randint(a, b)`, donde `a` y `b` son los límites inferior y superior del rango, respectivamente.

Aquí tienes un ejemplo de cómo usar `random.randint`:

```python
import random

# Generar un número aleatorio entre 1 y 10
numero_aleatorio = random.randint(1, 10)
print(numero_aleatorio)
```

Además de `random.randint`, la biblioteca `random` ofrece otras funciones útiles para generar números aleatorios:

- `random.random()`: Devuelve un número flotante aleatorio entre 0.0 y 1.0.
- `random.uniform(a, b)`: Devuelve un número flotante aleatorio entre `a` y `b`.
- `random.choice(seq)`: Devuelve un elemento aleatorio de una secuencia (como una lista o una cadena).

Aquí tienes ejemplos de estas funciones:

```python
import random

# Número flotante aleatorio entre 0.0 y 1.0
numero_flotante = random.random()
print(numero_flotante)

# Número flotante aleatorio entre 1.5 y 6.5
numero_uniforme = random.uniform(1.5, 6.5)
print(numero_uniforme)

# Elemento aleatorio de una lista
lista = [1, 2, 3, 4, 5]
elemento_aleatorio = random.choice(lista)
print(elemento_aleatorio)
```

Para más información sobre la biblioteca `random`, puedes consultar la [documentación oficial de Python](https://docs.python.org/3/library/random.html).

### Define what tools are we going to give to our brain agent

Go to `common/utils.py` to check the tools definition and the instructions on what tool to use when

In [20]:
tools = [www_search, sql_search, doc_search, book_search, chatgpt_search]

**Note**: Notice that since both the CSV file and the SQL Database have the same exact data, we are only going to use the SQLDBTool since it is faster and more reliable

# Option 1: Using OpenAI functions as router

We need a method to route the question to the right tool, one simple way to do this is to use OpenAI models functions via the Tools API (models 1106 and newer). To do this, we need to bind these tools/functions to the model and let the model respond with the right tool to use.

The advantage of this option is that there is no another agent in the middle between the experts (agent tools) and the user. Each agent tool responds directly. Also, another advantage is that multiple tools can be called in parallel.

**Note**: on this method it is important that each agent tool has the same system profile prompt so they adhere to the same reponse guidelines.

In [21]:
llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}

In [22]:
def call_tool(tool_invocation: dict) -> Union[str, Runnable]:
    """Function for dynamically constructing the end of the chain based on the model-selected tool."""
    tool = tool_map[tool_invocation["type"]]
    return RunnablePassthrough.assign(output=itemgetter("args") | tool)

def print_response(result: List):
    for answer in result:
        printmd("**"+answer["type"] + "**" + ": " + answer["output"])
        printmd("----")
    
# .map() allows us to apply a function to a list of inputs.
call_tool_list = RunnableLambda(call_tool).map()
agent = llm_with_tools | JsonOutputToolsParser() | call_tool_list

In [23]:
result = agent.invoke("Who is the current president of France?")
print_response(result)

Tool: bing
Agent Action: 
Invoking: `Searcher` with `{'query': 'current president of France 2023'}`



Agent Action: 
Invoking: `WebFetcher` with `{'url': 'https://www.elysee.fr/en/emmanuel-macron'}`



Agent Action: 
Invoking: `Searcher` with `{'query': 'current president of France site:elysee.fr'}`





**bing**: The current President of France is Emmanuel Macron. He was first elected on May 7, 2017, and re-elected on April 24, 2022 [[1]](https://www.elysee.fr/en/emmanuel-macron).

----

In [24]:
result = agent.invoke("docsearch,bing, what is CLP?")
print_response(result)

Tool: docsearch
Tool: bing
Agent Action: 
Invoking: `docsearch` with `{'query': 'CLP'}`



Agent Action: 
Invoking: `Searcher` with `{'query': 'CLP'}`



Agent Action: 
Invoking: `WebFetcher` with `{'url': 'https://www.clp.com.hk/en/index'}`



Agent Action: 
Invoking: `WebFetcher` with `{'url': 'https://www.clpgroup.com/en/index.html'}`



Agent Action: 
Invoking: `Searcher` with `{'query': 'site:clp.com.hk'}`





**docsearch**: El término "CLP" puede referirse a varios conceptos dependiendo del contexto. Aquí hay algunos ejemplos basados en los documentos encontrados:

1. **Proteasa 3C-Like (3CLpro)**: En el contexto de los virus, como el SARS-CoV, la proteasa 3C-Like (3CLpro) es una enzima crucial para el procesamiento proteolítico de los polipéptidos replicativos del virus. Esta enzima es un objetivo prometedor para el desarrollo de fármacos anti-SARS debido a su papel esencial en la replicación viral [[source]](https://onlinelibrary.wiley.com/doi/pdfdirect/10.1110/ps.052007306?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-01-30T14:55:42Z&st=2024-07-10T06:55:42Z&spr=https&sig=NQgAnpUqrSUPKdOZwtvdSQP2pjwoeUK0xlNtd%2F554t8%3D).

2. **Tripeptidil-peptidasa I (CLN2)**: En el contexto de enfermedades neurodegenerativas, la tripeptidil-peptidasa I, también conocida como CLN2, es una enzima cuya deficiencia en humanos conduce a la ceroidolipofuscinosis neuronal infantil tardía, una enfermedad neurodegenerativa fatal. Esta enzima es altamente conservada y ampliamente distribuida entre los organismos superiores [[source]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC280685/?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-01-30T14:55:42Z&st=2024-07-10T06:55:42Z&spr=https&sig=NQgAnpUqrSUPKdOZwtvdSQP2pjwoeUK0xlNtd%2F554t8%3D).

3. **Proteasa 3CL en PRRSV**: En el contexto del virus del síndrome reproductivo y respiratorio porcino (PRRSV), la proteasa 3CL juega un papel crucial en la maquinaria de transcripción y replicación del virus. Esta enzima ha sido clonada y sobreexpresada en Escherichia coli para estudios de cristalización [[source]](http://europepmc.org/articles/pmc2335157?pdf=render?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-01-30T14:55:42Z&st=2024-07-10T06:55:42Z&spr=https&sig=NQgAnpUqrSUPKdOZwtvdSQP2pjwoeUK0xlNtd%2F554t8%3D).

Estos son solo algunos ejemplos de cómo se puede interpretar "CLP" en diferentes contextos científicos.

----

**bing**: CLP (China Light and Power) is a major electricity company with a significant presence in Hong Kong and other regions. Here are some key details about CLP:

### Overview
- **CLP Power Hong Kong Limited** is a wholly-owned subsidiary of CLP Holdings Limited. Founded in 1901, it provides electricity to more than 80% of Hong Kong’s population [[1]](https://www.clp.com.hk/en/index).
- CLP Holdings Limited is a listed company in Hong Kong and one of the largest investor-owned power businesses in Asia. It operates as a vertically integrated power utility covering power generation, transmission, and distribution [[2]](https://www.clp.com.hk/en/about-clp).

### Services and Operations
- **Power Generation**: CLP generates electricity through various means, including renewable energy sources.
- **Power Transmission and Distribution**: The company ensures the reliable delivery of electricity to its customers.
- **Customer Services**: CLP offers online services for billing, payment, and customer support. They also have a rewards program and provide energy-saving tips [[3]](https://www.clp.com.hk/en/help-support).

### Additional Information
- **Website**: For more detailed information, you can visit their official website [here](https://www.clp.com.hk/en/index).

If you need more specific details or have any other questions, feel free to ask!

----

# Option 2: Using a user facing agent that calls the agent tools experts

With this method, we create a user facing agent that talks to the user and also talks to the experts (agent tools)

### Initialize the brain agent

In [34]:
agent = create_openai_tools_agent(llm, tools, CUSTOM_CHATBOT_PROMPT)

In [35]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [36]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [37]:
brain_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [38]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
print(random_session_id, ramdom_user_id)

session118 user356


### Let's talk to our GPT Smart Search Engine chat bot now

In [39]:
printmd(brain_agent_executor.invoke({"question": "tell me the formula in physics for momentum"}, config=config)["output"])

In physics, the formula for momentum (\( p \)) is given by:

\[ p = m \cdot v \]

where:
- \( p \) is the momentum,
- \( m \) is the mass of the object,
- \( v \) is the velocity of the object.

Momentum is a vector quantity, which means it has both magnitude and direction. The direction of the momentum vector is the same as the direction of the velocity vector.

In [33]:
printmd(brain_agent_executor.invoke({"question": "docsearch, what is a NP-complete problem?"}, config=config)["output"])

Tool: docsearch
Agent Action: 
Invoking: `docsearch` with `{'query': 'NP-complete problem'}`





Un problema **NP-completo** es un tipo de problema en la teoría de la complejidad computacional que tiene dos características principales:

1. **NP (Nondeterministic Polynomial time)**: Un problema está en NP si una solución propuesta para el problema puede ser verificada como correcta o incorrecta en tiempo polinómico por una máquina de Turing determinista. En otras palabras, si se te da una solución, puedes verificar su corrección rápidamente.

2. **NP-duro**: Un problema es NP-duro si cualquier problema en NP puede ser transformado (reducido) en tiempo polinómico a este problema. Esto implica que resolver un problema NP-duro es al menos tan difícil como resolver cualquier otro problema en NP.

Un problema es NP-completo si cumple ambas condiciones anteriores: está en NP y es NP-duro. Esto implica que si se encuentra un algoritmo de tiempo polinómico para resolver cualquier problema NP-completo, entonces todos los problemas en NP pueden ser resueltos en tiempo polinómico, lo cual resolvería la famosa pregunta de si P = NP.

### Ejemplo de Problema NP-completo

Un ejemplo de un problema NP-completo es la predicción de estructuras de pseudonudos en el ARN, que es crucial en procesos virales y celulares. Los algoritmos prácticos para la predicción de estructuras de ARN, incluyendo clases restringidas de pseudonudos, sufren de tiempos de ejecución elevados y baja precisión para secuencias largas [[1]](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2853144/?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2025-01-30T14:55:42Z&st=2024-07-10T06:55:42Z&spr=https&sig=NQgAnpUqrSUPKdOZwtvdSQP2pjwoeUK0xlNtd%2F554t8%3D).

Para más detalles sobre problemas NP-completos y su relevancia en diferentes campos, puedes consultar la literatura especializada en teoría de la complejidad computacional y algoritmos.

In [46]:
printmd(brain_agent_executor.invoke({"question": "sqlsearch, How many people died of covid in Texas in 2021?"}, config=config)["output"])

Tool: sqlsearch
Agent Action: 
Invoking: `sql_db_list_tables` with `{}`



Agent Action: 
Invoking: `sql_db_schema` with `{'table_names': 'covidtracking'}`



Agent Action: 
Invoking: `sql_db_query_checker` with `{'query': "SELECT SUM(death) AS total_deaths FROM covidtracking WHERE state = 'TX' AND date LIKE '2021%'"}`



Agent Action: 
Invoking: `sql_db_query` with `{'query': "SELECT SUM(death) AS total_deaths FROM covidtracking WHERE state = 'TX' AND date LIKE '2021%'"}`





In 2021, a total of 76,573 people died of COVID-19 in Texas.

This information was obtained by querying the `covidtracking` table for the sum of the `death` column where the state is 'TX' and the date starts with '2021'. The SQL query used was:

```sql
SELECT SUM(death) AS total_deaths FROM covidtracking WHERE state = 'TX' AND date LIKE '2021%'
```

In [41]:
# This question although does not contain instructions for a tool, the brain agent decides what tool to use
printmd(brain_agent_executor.invoke({"question": "What's a good place to dine today in downtown Seoul?"}, config=config)["output"])

Tool: bing
Agent Action: 
Invoking: `Searcher` with `{'query': 'best places to dine in downtown Seoul'}`



Agent Action: 
Invoking: `Searcher` with `{'query': 'top restaurants in downtown Seoul 2023'}`



Agent Action: 
Invoking: `WebFetcher` with `{'url': 'https://www.tripadvisor.com/Restaurants-g294197-Seoul.html'}`



Agent Action: 
Invoking: `WebFetcher` with `{'url': 'https://www.eater.com/maps/best-seoul-restaurants-38'}`





Based on the search results, here are some highly recommended places to dine in downtown Seoul:

### 1. **Mosu**
- **Description**: Recently elevated to a three-star Michelin restaurant, Mosu offers an exquisite dining experience with a focus on innovative Korean cuisine.
- **Link**: [Michelin Guide Seoul 2023](https://www.finedininglovers.com/article/michelin-stars-seoul-2023)

### 2. **Jungsik**
- **Description**: A two-star Michelin restaurant known for its modern take on traditional Korean dishes.
- **Link**: [Michelin Guide Seoul 2023](https://guide.michelin.com/sg/en/article/travel/5-new-michelin-starred-restaurants-in-seoul-en-2023-sg)

### 3. **Mingles**
- **Description**: Another two-star Michelin restaurant that blends Korean and Western culinary techniques.
- **Link**: [Michelin Guide Seoul 2023](https://guide.michelin.com/sg/en/article/travel/5-new-michelin-starred-restaurants-in-seoul-en-2023-sg)

### 4. **TocToc**
- **Description**: A one-star Michelin restaurant offering a unique dining experience with a focus on seasonal ingredients.
- **Link**: [Michelin Guide Seoul 2023](https://guide.michelin.com/sg/en/article/travel/5-new-michelin-starred-restaurants-in-seoul-en-2023-sg)

### 5. **Balwoo Gongyang**
- **Description**: A one-star Michelin restaurant that specializes in temple cuisine, offering a serene and spiritual dining experience.
- **Link**: [Michelin Guide Seoul 2023](https://guide.michelin.com/sg/en/article/travel/5-new-michelin-starred-restaurants-in-seoul-en-2023-sg)

### 6. **Joo Ok**
- **Description**: Known for its creative and modern Korean dishes, Joo Ok is a one-star Michelin restaurant that promises a memorable dining experience.
- **Link**: [Michelin Guide Seoul 2023](https://guide.michelin.com/sg/en/article/travel/5-new-michelin-starred-restaurants-in-seoul-en-2023-sg)

### 7. **Gaon**
- **Description**: A three-star Michelin restaurant that offers a luxurious dining experience with a focus on traditional Korean cuisine.
- **Link**: [Michelin Guide Seoul 2023](https://guide.michelin.com/sg/en/article/travel/5-new-michelin-starred-restaurants-in-seoul-en-2023-sg)

For a more comprehensive list of dining options, you can also check out the following resources:
- [Tripadvisor's Best Restaurants in Seoul](https://www.tripadvisor.com/Restaurants-g294197-Seoul.html)
- [Eater's 38 Best Restaurants in Seoul](https://www.eater.com/maps/best-seoul-restaurants-38)

These sources provide detailed reviews and additional dining options to explore in downtown Seoul. Enjoy your meal!

# Option 3: Using LangGraph
See Notebook 11.5 (experimental)

# Summary

Great!, We just built the GPT Smart Search Engine!
In this Notebook we created the brain, the decision making Agent that decides what Tool to use to answer the question from the user. This is what was necessary in order to have an smart chat bot.

We can have many tools to accomplish different tasks, including connecting to APIs, dealing with File Systems, and even using Humans as Tools. For more reference see [HERE](https://python.langchain.com/docs/integrations/tools/)